In [1]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 50.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.de

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [3]:
from utils import *
from data_preperation import data_preperation
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
import os
import shutil
from jax import numpy as np

In [4]:
dataset = TUDataset(root="Masterarbeit", name="MUTAG")

# make lists of dense adjacency matrix, node feature array and y
edge_index = list()
graps_node_features = list()
ys = list()
for data in dataset:
    edge_index.append(data.edge_index)
    graps_node_features.append(np.array(data.x))
    ys.append(np.array(data.y))

In [5]:
# save preprocessed data
# creates the following files in the base path directory
# ys.npy
# graps_node_features.npy
# graphs_edge_features.npy
# graph_conv_pattern.npy
# two_wl_pattern_radius_1.npy, two_wl_pattern_radius_2.npy, ..., two_wl_pattern_radius_k.npy
two_wl_radius = [2]
dataset_name = "MUTAG"
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"
graps_edge_features = None
edge_featur_init = "BIAS"

if not os.path.exists(base_path):
    os.makedirs(base_path)

data_preperation(edge_index, graps_node_features, graps_edge_features, edge_featur_init, ys, dataset_name, base_path, two_wl_radius)

UnboundLocalError: ignored

In [4]:
# load preprocessed data

two_wl_radius = [1]
dataset_name = "MUTAG"
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"

ys = np.load(base_path + f"/ys.npy")
graps_node_features = np.load(base_path + f"/graps_node_features.npy")
graphs_edge_features = np.load(base_path + f"/graphs_edge_features.npy")
graph_conv_pattern = np.load(base_path + f"/graph_conv_pattern.npy")

two_wl_patterns = list()

for r in two_wl_radius:
    two_wl_patterns.append(np.load(base_path + f"/two_wl_pattern_radius_{r}.npy"))

print(two_wl_patterns[0].shape)
print(graphs_edge_features.shape)

(376, 4)
(188, 28, 28, 8)


In [ ]:
# clear the folder
shutil.rmtree("/content/drive/MyDrive/MasterarbeitData/Preprocessed/splits/MUTAG")

In [8]:
188 / 3

62.666666666666664

In [9]:
# split the preprocessed data into pieces, to save memory in the kernel calcualtion

size = 3
r = 62
dataset_size = graphs_edge_features.shape[0] 

base_path_splits = base_path + f"/splits/size_{size}_range_{r}" 
if not os.path.exists(base_path_splits):
    os.makedirs(base_path_splits)

for i in range(r+1):
  for j in range(r+1):
    
    if not i == r:
      x1_batch_range = list(range(i*size,(1+i)*size))
    else:
      x1_batch_range = list(range(i*size,dataset_size-1))

    if not j == r:
      x2_batch_range = list(range(j*size,(1+j)*size))
    else:
      x2_batch_range = list(range(j*size,dataset_size-1))
    
    np.save(base_path_splits + f"/graphs_edge_features_1_{i*size}_{(1+i)*size}", graphs_edge_features[x1_batch_range,:])
    np.save(base_path_splits + f"/graphs_edge_features_2_{j*size}_{(1+j)*size}", graphs_edge_features[x2_batch_range,:])

    for r, p in zip(two_wl_radius, two_wl_patterns):
      
      np.save(base_path_splits + f"/two_wl_pattern_radius_{r}_1_{i*size}_{(1+i)*size}", p[column_in_values(p[:,0], np.array(x1_batch_range)),:])
      np.save(base_path_splits + f"/two_wl_pattern_radius_{r}_2_{j*size}_{(1+j)*size}", p[column_in_values(p[:,0], np.array(x2_batch_range)),:])

